In [ ]:
%matplotlib notebook
import ipywidgets, pylab as pl
from ImageD11 import transformer

In [ ]:
# input data from peaksearch and starting parameter file:
trans=transformer.transformer()
trans.loadfiltered("github/FABLE-3DXRD/ImageD11/test/nac_demo/peaks.out_merge_t200")
trans.loadfileparameters("github/FABLE-3DXRD/ImageD11/test/nac_demo/nac.prm")

In [ ]:
def parCallBack( arg ):
    trans.parameterobj.parameters.update( { arg['owner'].description : arg['new'] } )
    drawPlot()
    
def fixVaryCallBack( arg ):
    name = arg['owner'].description.split(" ")[1]
    vars = trans.getvars()
    if arg.new and not (name in vars):
        vars.append( name )
    if name in vars and not arg.new:
        vars.remove(name)
    trans.parameterobj.set_varylist(vars)
    
def fitCallBack(arg):
    """ fit call back - runs fit """
    lo, hi = ax1.get_xlim()
    trans.fit( lo, hi )
    for i, pname in enumerate(vars):
        layout[i,0].value = trans.parameterobj.get(pname)
    drawPlot()
    
def drawPlot():
    tth, eta = trans.compute_tth_eta()
    pt1.set_data( tth, eta )

# Things to be edited in the UI
vars = "y_center z_center distance tilt_y tilt_z".split()
steps = (1,         1,       100,     0.01, 0.01, 0)
nv = len(vars)

# Draw the widgets:
layout = ipywidgets.GridspecLayout(nv+1,2)
for i,( pname, pstep ) in enumerate( zip( vars, steps ) ) :
    layout[i,0] = ipywidgets.FloatText( description=pname, 
        value = trans.parameterobj.parameters.get(pname),
        step=pstep)
    layout[i,0].observe( parCallBack , names='value' )
    layout[i,1] = ipywidgets.ToggleButton( description="Vary "+pname, 
        value = pname in trans.getvars() )
    layout[i,1].observe( fixVaryCallBack, names='value' )
    
layout[nv,0] = ipywidgets.FloatText( description='fit_tolerance', 
        value = trans.parameterobj.parameters.get("fit_tolerance"), step=0,)
layout[nv,0].observe( parCallBack , names='value' )

layout[nv,1] = ipywidgets.Button(description="Run Fit (blocks)")
layout[nv,1].on_click( fitCallBack )

In [ ]:
# Draw plot
fig1 = pl.figure(1, figsize=(9,6))
ax1 = fig1.add_subplot()
tth, eta = trans.compute_tth_eta()
trans.addcellpeaks()
pt1, = ax1.plot( tth, eta, ",")
ax1.set(xlabel="tth", ylabel="eta")
ax1.plot( trans.theorytth, [0,]*len(trans.theorytth), "r|", ms=360, alpha=0.2 )
# Add controls
display(layout)